<a href="https://colab.research.google.com/github/YasuharuSuzuki/25_isco3/blob/main/" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 01_データ前処理

## scikit-learn
<img src="https://raw.githubusercontent.com/scikit-learn/scikit-learn/main/doc/logos/scikit-learn-logo.png">

- 前回の授業ではワインの種類やあやめの品種を学習・予測する「分類モデル」を作成しました。
- 今回の授業では引き続きscikit-learnを使用して、数値予測を行う「回帰モデル」を作成してみましょう。

---

### 🌐 参考サイト
#### [【Python入門】7-2. scikit-learnライブラリ](https://utokyo-ipp.github.io/7/7-2.html)

---
---

### 📑 使用するデータセットについて
#### 👩🏻‍🍳 レストランチップ額データセット
- seaborn に同梱されているレストランでの会計とチップに関する実データをまとめたデータセットです。
- 来客者の情報（性別、喫煙の有無、曜日、食事の時間、同席人数、支払額）から、チップの額を予測する回帰モデルを作ることが出来ます。
- 後述しますが、カテゴリカル変数が含まれております。
- また、欠損値が存在します（元々のデータセットにはありませんが、授業用に加工しています）

##### 📈 カラム名と内容
| No. | カラム名 | 内容 |
| --- | --- | --- |
| 1 | total_bill | 総支払額(食事代、税込み)　(USドル) |
| 2 | sex | 性別 | 
| 3 | smoker | 喫煙者か否か |
| 4 | day | 曜日（Thur / Fri / Sat / Sun） |
| 5 | time | 食事の時間 |
| 6 | size | 同席人数 |
| 7 | tip | **チップ額（USドル）。目的変数** | 


---

#### 💉 医療保険料データセット
- Kaggle 等で配信されている個人属性から医療保険の保険料（医療費）を予測するための機械学習用データセットです。
- 個人情報（年齢、性別、bmi、子供の数、喫煙の有無、居住地域）から、年間の保険料の額を予測する回帰モデルを作ることが出来ます。
- こちらもカテゴリカル変数が含まれております。
- 欠損値も存在します（元々のデータセットにはありませんが、授業用に加工しています）

##### 📈 カラム名と内容
| No. | カラム名 | 内容 |
| --- | --- | --- |
| 1 | age | 年齢 |
| 2 | sex | 性別(male / female) | 
| 3 | bmi | BMI(体格指数) | 
| 4 | children | 扶養している子供の数 | 
| 5 | smoker | 喫煙者か否か(yes / no) |
| 6 | region | 居住地域（northeast / northwest / southeast / southwest） |
| 7 | charges | **年間医療保険料（目的変数）** |

##### LICENSE
- [Medical Cost Personal Datasets](https://www.kaggle.com/datasets/mirichoi0218/insurance)
- [Licensed under the Database Contents License (DbCL) v1.0](https://opendatacommons.org/licenses/dbcl/1-0/)

---
---

### 🍢 データ前処理について
- **データ前処理** とは、「集めたデータを、そのまま使うのではなく、**分析や機械学習で使いやすい形に整える作業** 」のことです。

現実のデータは、そのままでは次のような問題を含んでいることが多くあります。
  - 入力されていない値（欠損値）がある
  - 表記ゆれがある（例：「東京」「東京都」「TOKYO」）
  - 数値として扱えないデータが混ざっている
  - 明らかにおかしな値（外れ値）が含まれている

このような状態のまま分析を行うと、 **正しい結果が得られない** ため、事前にデータを整える必要があります。

---

### ⬇️ ライブラリのインストール

In [ ]:
# matplotlib日本語対応ライブラリのインストール
!pip install matplotlib-fontja

### 📁 GoogleDrive の Mount

In [ ]:
# drive.mount()を使用してGoogleDriveをMount
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# os.chdir()を使用してカレントディレクトリを変更。作成したショーットカット名を使ってパスを指定します。
import os
os.chdir('/content/drive/MyDrive/＜作成したショートカット名＞')

### ➜]　ライブラリのimport

In [2]:
import numpy as np # 数値演算ライブラリ
import pandas as pd # データ分析ライブラリ
import matplotlib.pyplot as plt # 可視化ライブラリ
import seaborn as sns # 可視化ライブラリ
import joblib # モデル保存・読み込みライブラリ
from sklearn.preprocessing import LabelEncoder # 文字データを数字に置換するクラス（後述）
from sklearn.model_selection import train_test_split # scikit-learnの訓練データ / テストデータ 分割関数
from sklearn import linear_model # scikit-learnのlinear_modelライブラリ
from sklearn.ensemble import RandomForestRegressor  # RandomForestRegressor (回帰モデル)
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score  # 回帰モデルの性能評価用関数

### ✍️ 定数の定義
- データファイルパスを定義しておきます
- データ読み込み・データ書き込みの際に使用してください。修正は不要です

In [3]:
# データファイルパス定義
TIPS_CSV = "./data/tips_customized.csv"
MEDICAL_COST_CSV = "./data/medical_cost_customized.csv"

# 前処理後の出力ファイルパス定義
TIPS_PREPROCESSED_CSV = "./data/tips_preprocessed.csv"
MEDICAL_PREPROCESSED_CSV = "./data/medical_preprocessed.csv"

# RANDOM SEED 定義
RANDOM_SEED = 0

---
---
---

### 🍽️ サンプルプログラム1　レストランチップ額データセットの値の確認

以下の手順で、レストランチップ額データセットの値を確認します。
- チップ額のデータセットを読み込む
- head() 読み込んだDataFrameをざっと確認する
- value_counts() sex列の確認 
- value_counts() smoker列の確認
- value_counts() day列の確認
- value_counts() time列の確認
- info() 行数、列数、データ型、欠損値の確認
- describe() 基本統計量の確認

In [4]:
# データセットを読み込み
df_tips = pd.read_csv(TIPS_CSV, index_col=0) # 0番の列をindexに使用するため、index_col=0を指定します

In [5]:
# 読み込んだDataFrameを確認
# 先頭5行をざっと眺める
df_tips.head()

,total_bill,sex,smoker,day,time,size,tip
0,16.99,Female,No,Sun,Dinner,2,1.01
1,10.34,Male,No,Sun,Dinner,3,1.66
2,21.01,Male,No,Sun,Dinner,3,3.50
3,23.68,Male,No,Sun,Dinner,2,3.31
4,24.59,Female,No,Sun,Dinner,4,3.61


- これまでと違ってMale/Femaleなどの文字列が含まれています。

In [6]:
# sex列の確認
df_tips['sex'].value_counts()

sex
Male      157
Female     87
Name: count, dtype: int64

- 性別は男性/女性のみ

In [7]:
# smoker列の確認
df_tips['smoker'].value_counts()

smoker
No     146
Yes     88
Name: count, dtype: int64

- 喫煙者かどうかも２値のみです

In [8]:
# day列の確認
df_tips['day'].value_counts()

day
Sat     87
Sun     76
Thur    62
Fri     19
Name: count, dtype: int64

- dayには曜日が入っていて、木曜日、金曜日、土曜日、日曜日のみが入っています（月曜日～水曜日は存在しない）

In [9]:
# time列の確認
df_tips['time'].value_counts()

time
Dinner    176
Lunch      68
Name: count, dtype: int64

- dinnerが多い

##### 🗂️ カテゴリカル変数
- 特定のグループまたは名目上のカテゴリに、いくつかの質的特性に基づいて値を割り当てた変数のことをカテゴリカル変数と言います。
- 質的変数とも言います。
- 上記のsex, smoker, dayはそれぞれカテゴリカル変数と言えます。
- SVRやRandomForestRegressorのような回帰モデルにカテゴリ変数を指定するとエラーが発生するので対策が必要です。（後ほど扱います）

### 💬 行数、列数、データ型、欠損値の確認

In [10]:
# info() 行数、列数、データ型、欠損値の確認
df_tips.info()

<class 'pandas.core.frame.DataFrame'>
Index: 244 entries, 0 to 243
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   total_bill  244 non-null    float64
 1   sex         244 non-null    object 
 2   smoker      234 non-null    object 
 3   day         244 non-null    object 
 4   time        244 non-null    object 
 5   size        244 non-null    int64  
 6   tip         244 non-null    float64
dtypes: float64(2), int64(1), object(4)
memory usage: 15.2+ KB


- smoker の変数が欠損しています

##### 👻 欠損値
- データセット内のある変数がなんらかの理由によりデータが欠落しているものを刺します。
- 例えばアンケート調査で無回答だった場合や、センサーデータで取得に失敗したデータなどが該当します。
- モデルに欠損値が含まれたデータを指定するとエラーが発生するため、対策が必要です。（後ほど扱います）

### 📊 基本統計量の確認

In [11]:
# describe() 基本統計量の確認
df_tips.describe()

,total_bill,size,tip
count,244.000000,244.000000,244.000000
mean,19.785943,2.569672,2.998279
std,8.902412,0.951100,1.383638
min,3.070000,1.000000,1.000000
25%,13.347500,2.000000,2.000000
50%,17.795000,2.000000,2.900000
75%,24.127500,3.000000,3.562500
max,50.810000,6.000000,10.000000


- total_billおよびtipでは、75%タイルからmax値に大きな開きが発生しており、外れ値か、べき分布のデータであることが考えられます。

---

### 💉 練習プログラム1　医療保険料データセットの値の確認 (15点)

以下の手順で、医療保険料データセットの値を確認し、それぞれ考察を付けてみましょう。  
1️⃣ 医療保険料のデータセットを読み込む  
2️⃣ head() 読み込んだDataFrameをざっと確認する   
3️⃣ value_counts() sex列の確認   
4️⃣ value_counts() smoker列の確認  
5️⃣ value_counts() region列の確認  
6️⃣ info() 行数、列数、データ型、欠損値の確認  
7️⃣ describe() 基本統計量の確認  

<!-- PROBLEM_TYPE: SYNTAX_EXERCISE -->

In [12]:
# データセットを読み込み
df_medical = pd.read_csv(MEDICAL_COST_CSV, index_col=0) # 0番の列をindexに使用するため、index_col=0を指定します

# 先頭５行のデータをざっと眺める
df_medical.head()

# 考察：カテゴリカル変数が含まれています

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


In [13]:
# value_counts() sex列の確認 
df_medical['sex'].value_counts()

# 考察：性別は男性/女性のみ

sex
male      676
female    662
Name: count, dtype: int64

In [14]:
# value_counts() smoker列の確認
df_medical['smoker'].value_counts()

# 考察：喫煙者かどうかも2値のみです

smoker
no     1061
yes     273
Name: count, dtype: int64

In [15]:
# value_counts() region列の確認
df_medical['region'].value_counts()

# 考察：regionは4種類のみ

region
southeast    364
southwest    325
northwest    325
northeast    324
Name: count, dtype: int64

In [16]:
# info() 行数、列数、データ型、欠損値の確認
df_medical.info()

# 考察
# - smoker の変数が少しだけ欠損しています
# - レストランのチップ予測と比較すると、1000件を超えておりデータ数が多い点は良い傾向

<class 'pandas.core.frame.DataFrame'>
Index: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   object 
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1334 non-null   object 
 5   region    1338 non-null   object 
 6   charges   1338 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 83.6+ KB


In [ ]:
# describe() 基本統計量の確認
df_medical.describe()

# 考察
# - bmiおよびchargesでは、75%タイルからmax値に大きな開きが発生しており、外れ値か、べき分布のデータであることが考えられます。

In [ ]:
#@title 送信処理用共通プログラム（以下のセルを「▶ 実行」して下さい。） { display-mode: "form" }
#@markdown 解答の送信に必要です。（中身は絶対に修正しないでください）

import os
os.makedirs(".client/python", exist_ok=True)  # 隠しフォルダを作成
!wget -q https://raw.githubusercontent.com/YasuharuSuzuki/25_isco3/main/91_notebook_client/src/setup.sh -O .client/setup.sh   # 外部コードをダウンロードして隠しフォルダに配置
!chmod +x ./.client/setup.sh
!bash ./.client/setup.sh # 実行

# sandbox実行用
# os.symlink("../../src", ".client", target_is_directory=True)
# !bash ./sandbox_setup.sh # 実行

# モジュールの初期化
exec(open('.client/client_setup.py').read())

# ノートブック固有の設定
NOTEBOOK_PATH = "05_機械学習(回帰)/01_データ前処理.ipynb"
set_notebook_config(NOTEBOOK_PATH)

In [ ]:
#@title 問1 送信ボタン表示（以下のセルを「▶ 実行」して下さい。） { display-mode: "form" }
#@markdown メールアドレスを取得して、送信ボタンを押してください（中のプログラムは絶対に修正しないでください）
display(create_submit_button(problem_number=1, button_name="練習プログラム"))

---
---
---

### 👻 サンプルプログラム2　欠損値の対策

チップ額データセットに、以下の手順で欠損値の対策を行います。

- isnull().sum() 欠損値の数を確認
- isnull().any() 欠損値を持つ行を確認
- dropna() 欠損値の削除
- info() 欠損値の除去を確認

In [23]:
# 欠損値の数を確認
# isnull()という関数を使用して欠損値かどうかを取得し、sum()関数を用いて集計することが可能です
df_tips.isnull().sum()

total_bill     0
sex            0
smoker        10
day            0
time           0
size           0
tip            0
dtype: int64

- smoker 列に10件のみ存在するようです
- 量としては多くないので、そのまま削除しても良さそうです

In [24]:
# 欠損値を持つ行を確認
# any()関数を用いると欠損行を抽出できます
df_tips[df_tips.isnull().any(axis=1)]

,total_bill,sex,smoker,day,time,size,tip
9,14.78,Male,NaN,Sun,Dinner,2,3.23
40,16.04,Male,NaN,Sat,Dinner,3,2.24
106,20.49,Male,NaN,Sat,Dinner,2,4.06
140,17.47,Female,NaN,Thur,Lunch,2,3.50
153,24.55,Male,NaN,Sun,Dinner,4,2.00
171,15.81,Male,NaN,Sat,Dinner,2,3.16
174,16.82,Male,NaN,Sun,Dinner,2,4.00
186,20.90,Female,NaN,Sun,Dinner,3,3.50
215,12.90,Female,NaN,Sat,Dinner,2,1.10
242,17.82,Male,NaN,Sat,Dinner,2,1.75


- 目的変数であるtip値を確認。2σ以上の値になっていないため、削除しても影響は少なめと思われる。
- Dinnerが多いが、元々Dinnerの方が多めのため、それほど影響は無さそう

In [25]:
# 欠損値の削除
# dropna()メソッドを使用して欠損値を削除する
df_tips2 = df_tips.dropna(how='any')  # how='any'は全ての列のうちどれか１つでも欠損値を含む行を対象とすることを指定しています
df_tips2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 234 entries, 0 to 243
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   total_bill  234 non-null    float64
 1   sex         234 non-null    object 
 2   smoker      234 non-null    object 
 3   day         234 non-null    object 
 4   time        234 non-null    object 
 5   size        234 non-null    int64  
 6   tip         234 non-null    float64
dtypes: float64(2), int64(1), object(4)
memory usage: 14.6+ KB


- 欠損値を削除した。
- ちなみに削除する以外の方法としては、欠損値を予測するモデルを作り、値を埋めて活かす方法も存在する

---

### 🗿 練習プログラム2 欠損値の対策(10点)

医療保険料のデータセットに以下の手順で欠損値の対策を行い、それぞれ考察を付けてみましょう。    
※読み込み済みのDataFrameを活用しましょう。  

1️⃣ isnull().sum() 欠損値の数を確認  
2️⃣ isnull().any() 欠損値を持つ行を確認  
3️⃣ dropna() 欠損値の削除  
4️⃣ info() 欠損値の除去を確認  

<!-- PROBLEM_TYPE: SYNTAX_EXERCISE -->

In [19]:
# 欠損値の数を確認
# isnull().sum() で確認します。
df_medical.isnull().sum()

# 考察
# smoker 列に4件のみ存在するようです
# 量としては多くないので、そのまま削除しても良さそうです

age         0
sex         0
bmi         0
children    0
smoker      4
region      0
charges     0
dtype: int64

In [20]:
# 欠損値を持つ行を確認
# any()関数を用いると欠損行を抽出できます
df_medical[df_medical.isnull().any(axis=1)]

# 考察
# 目的変数であるcharges値を確認。2σ内のため削除しても影響は少なめと思われる。

,age,sex,bmi,children,smoker,region,charges
32,19,female,28.600,5,NaN,southwest,4687.79700
347,46,male,33.345,1,NaN,northeast,8334.45755
429,27,female,30.400,3,NaN,northwest,18804.75240
827,36,male,28.025,1,NaN,northeast,20773.62775


In [21]:
# 欠損値の削除
# dropna()メソッドを使用して欠損値を削除する
df_medical2 = df_medical.dropna(how='any')  # how='any'は全ての列のうちどれか１つでも欠損値を含む行を対象とすることを指定しています
df_medical2.info()

# 欠損値の削除を確認

<class 'pandas.core.frame.DataFrame'>
Index: 1334 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1334 non-null   int64  
 1   sex       1334 non-null   object 
 2   bmi       1334 non-null   float64
 3   children  1334 non-null   int64  
 4   smoker    1334 non-null   object 
 5   region    1334 non-null   object 
 6   charges   1334 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 83.4+ KB


In [ ]:
#@title 問2 送信ボタン表示（以下のセルを「▶ 実行」して下さい。） { display-mode: "form" }
#@markdown メールアドレスを取得して、送信ボタンを押してください（中のプログラムは絶対に修正しないでください）
display(create_submit_button(problem_number=2, button_name="練習プログラム"))

---
---
---

### 🗂️ サンプルプログラム3　カテゴリカル変数の対策とデータ保存

以下の手順でチップ額データセットのカテゴリカル変数の対策を行います。

- カテゴリカル変数の対象とする列名をリストで定義
- for文でループしながら、各列をLabelEncoderを使用して変換
- info() を使用して、対策後のデータ型を確認
- 前処理を行ったデータをto_csv()でファイルに保存する(TIPS_PREPROCESSED_CSVで定義済みのファイル名を使用する)

In [27]:
categories = ['sex','smoker','day','time']  # カテゴリカル変数の対象とする列を指定する

df_tips3 = df_tips2.copy()  # DataFrameを書き換えるため、変換する
for category in categories:
    label_encoder = LabelEncoder()  # 文字列を数値に変換するクラスのインスタンスを作成
    label_encoder = label_encoder.fit(df_tips3[category])  # fit関数で文字列と数値の関係を取得
    df_tips3[category] = label_encoder.transform(df_tips3[category])  # transformで変換を実行
    df_tips3[category] = df_tips3[category].astype('category') # カテゴリカル変数には専用の型'category'があるため、型変換しておく
    print(df_tips3[category].value_counts())  # 変換後の値を表示

sex
1    150
0     84
Name: count, dtype: int64
smoker
0    146
1     88
Name: count, dtype: int64
day
1    82
2    72
3    61
0    19
Name: count, dtype: int64
time
0    167
1     67
Name: count, dtype: int64


In [28]:
# info() を使用して、カテゴリ変数対策後の型を表示
df_tips3.info()

<class 'pandas.core.frame.DataFrame'>
Index: 234 entries, 0 to 243
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   total_bill  234 non-null    float64 
 1   sex         234 non-null    category
 2   smoker      234 non-null    category
 3   day         234 non-null    category
 4   time        234 non-null    category
 5   size        234 non-null    int64   
 6   tip         234 non-null    float64 
dtypes: category(4), float64(2), int64(1)
memory usage: 8.8 KB


- object型（文字列）が無くなり、categoryになった

In [ ]:
# 前処理を行ったデータの保存
df_tips3.to_csv(TIPS_PREPROCESSED_CSV)

### 🛍️ 練習プログラム3　カテゴリカル変数の対策とデータ保存 (5点)

以下の手順で医療保険料データセットのカテゴリカル変数の対策を行い、データを保存しましょう。  
※読み込み済みのDataFrameを活用しましょう。  

- カテゴリカル変数の対象とする列名をリストで定義
- for文でループしながら、各列をLabelEncoderを使用して変換
- info() を使用して、対策後のデータ型を確認
- 前処理を行ったデータをto_csv()でファイルに保存する(MEDICAL_PREPROCESSED_CSVで定義済みのファイル名を使用する)

<!-- PROBLEM_TYPE: SYNTAX_EXERCISE -->

In [30]:
categories = ['sex','smoker','region']  # カテゴリカル変数の対象とする列を指定する

df_medical3 = df_medical2.copy()  # DataFrameを書き換えるため、変換する
for category in categories:
    label_encoder = LabelEncoder()  # 文字列を数値に変換するクラスのインスタンスを作成
    label_encoder = label_encoder.fit(df_medical3[category])  # fit関数で文字列と数値の関係を取得
    df_medical3[category] = label_encoder.transform(df_medical3[category])  # transformで変換を実行
    df_medical3[category] = df_medical3[category].astype('category') # カテゴリカル変数には専用の型'category'があるため、型変換しておく
    print(df_medical3[category].value_counts())  # 変換後の値を表示

sex
1    674
0    660
Name: count, dtype: int64
smoker
0    1061
1     273
Name: count, dtype: int64
region
2    364
1    324
3    324
0    322
Name: count, dtype: int64


In [31]:
# info() を使用して、カテゴリ変数対策後の型を表示
df_medical3.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1334 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   age       1334 non-null   int64   
 1   sex       1334 non-null   category
 2   bmi       1334 non-null   float64 
 3   children  1334 non-null   int64   
 4   smoker    1334 non-null   category
 5   region    1334 non-null   category
 6   charges   1334 non-null   float64 
dtypes: category(3), float64(2), int64(2)
memory usage: 56.5 KB


In [32]:
# 前処理を行ったデータの保存
df_medical3.to_csv(MEDICAL_PREPROCESSED_CSV)

In [ ]:
#@title 問3 送信ボタン表示（以下のセルを「▶ 実行」して下さい。） { display-mode: "form" }
#@markdown メールアドレスを取得して、送信ボタンを押してください（中のプログラムは絶対に修正しないでください）
display(create_submit_button(problem_number=3, button_name="練習プログラム"))